In [12]:
import os
from dotenv import load_dotenv, find_dotenv

import httpx
import pandas as pd

load_dotenv(find_dotenv())
pd.options.display.max_colwidth = 1500

In [13]:
def get_token():
    response = httpx.post(
        str(os.getenv("TOKEN_ENDPOINT")),
        data={
            "client_id": str(os.getenv("CLIENT_ID")),
            "client_secret": str(os.getenv("CLIENT_SECRET")),
            "scope": str(os.getenv("SCOPE")),
            "grant_type": "client_credentials",
        },
        proxies={
            "http://": httpx.Proxy(str(os.getenv("FORD_PROXY"))),
            "https://": httpx.Proxy(str(os.getenv("FORD_PROXY"))),
        },
        timeout=160,
    )
    print(
        "token expires in:",
        round(int(response.json()["expires_in"]) / 60, 0),
        "minutes",
    )
    return response.json()["access_token"]

In [14]:
def call_api_movement(prompt: str, token: str):
    content = {
        "model": "gpt-4",
        "context": "Analyse this complaint and answer this following question",  # sets the overall behavior of the assistant.
        "messages": [
            {
                "role": "user",
                "content": prompt,
            }
        ],
        "parameters": {
            "temperature": 0.05,  # Determines the randomnes of the model's response.
        },
    }
    try:
        response = httpx.post(
            str(os.getenv("API_ENDPOINT")),
            headers={"Authorization": f"Bearer {token}"},
            json=content,
            timeout=20,
        )
        message = response.json()["content"]
        print(message)
        if "\n" in message:
            message = message.split("\n")[0]

        return message

    except httpx.HTTPError as exc:
        print(exc)

### DO YOUR ANALYSIS BEYOND THIS COMMENT 

In [15]:
df = pd.read_csv("../../../../data/raw/recalls_mock.csv")
df["to be binned"] = (
    f'With this problematic component {str(df["COMPONENT"])} '
    + f'and this decription: {str(df["RECALL_DESCRIPTION"])}.'
)

In [17]:
from ....utils.funtions import load_categories


token = get_token()
parts = (
    "door, window, windshield, wiper, glass, hood, trunk, moonroof, "
    + "bumper, tail light, pillar, undershield, roof rack, latch, he"
    + "adlight, door handle, door keypad, window, weatherstripping, "
    + "side mirror, lighting, swing gate, cowl grille, hard top, ski"
    + "d plate, sheet metal, running boards, water leak, etc"
)
classes = list(load_categories("Failures"))
content = (
    "Question 1: For this description, check if it's related to an e"
    + f"xternal part of the car, body exterior, ({parts}). If yes, a"
    + "nswer 'F8'. IF not, answer 'NOT F8'. Note that most of the pr"
    + "oblems related to power liftgate electrical problems and rear"
    + " view camera are NOT F8. Question 2: For each of these senten"
    + "ces that your answer 1 was 'F8', check if it is related to on"
    + f"ly one of the following categories: {classes}. You should gi"
    + "ve only one answer with one answer for Question 1 and one ans"
    + "2er for Question 2 in the following format: 'ANSWER 1~~~ANSWE"
    + "R 2'. Note: 'OWD' means 'opened while driving' and 'F&F' mean"
    + "s 'fit and finish', for problems related to flushness and mar"
    + "gin. Note 2: For model Escape (2020 forward), there is a comm"
    + "on problem related to door check arm when the decription is r"
    + "elated to the door making popping sounds, opening and closing"
    + " problens, hinges and welds. If you cannot relate, answer NOT"
    + " SURE. Answer in the correct order. If you cannot assist, ans"
    + "wer 1, and answer 2 must be NA. You should be objective and c"
    + "old. Never change the answer format mentioned."
)
df[["FUNCTION_", "COMPONENT", "FAILURE"]] = (
    df["to be binned"]
    .apply(lambda row: call_api_movement(f"{row} {content}", token))
    .to_list()
)
df[df["filter"] == "YES"].to_csv("Result.csv")

ImportError: attempted relative import with no known parent package